**756. Pyramid Transition Matrix**

**Medium**

**Companies**: Airbnb Google Uber Snapchat

You are stacking blocks to form a pyramid. Each block has a color, which is represented by a single letter. Each row of blocks contains **one less block** than the row beneath it and is centered on top.

To make the pyramid aesthetically pleasing, there are only specific **triangular patterns** that are allowed. A triangular pattern consists of a **single block** stacked on top of **two blocks**. The patterns are given as a list of three-letter strings allowed, where the first two characters of a pattern represent the left and right bottom blocks respectively, and the third character is the top block.

- For example, "ABC" represents a triangular pattern with a 'C' block stacked on top of an 'A' (left) and 'B' (right) block. Note that this is different from "BAC" where 'B' is on the left bottom and 'A' is on the right bottom.

You start with a bottom row of blocks bottom, given as a single string, that you **must** use as the base of the pyramid.

Given bottom and allowed, return true if you can build the pyramid all the way to the top such that **every triangular pattern** in the pyramid is in allowed, or false otherwise.

**Example 1:**

```
Input: bottom = "BCD", allowed = ["BCC","CDE","CEA","FFF"]
Output: true
```

**Explanation:** The allowed triangular patterns are shown on the right.

Starting from the bottom (level 3), we can build "CE" on level 2 and then build "A" on level 1.

There are three triangular patterns in the pyramid, which are "BCC", "CDE", and "CEA". All are allowed.

**Example 2:**

```
Input: bottom = "AAAA", allowed = ["AAB","AAC","BCD","BBE","DEF"]
Output: false
```

**Explanation:** The allowed triangular patterns are shown on the right.

Starting from the bottom (level 4), there are multiple ways to build level 3, but trying all the possibilites, you will get always stuck before building level 1.

**Constraints:**

- 2 <= bottom.length <= 6
- 0 <= allowed.length <= 216
- allowed[i].length == 3
- The letters in all input strings are from the set {'A', 'B', 'C', 'D', 'E', 'F'}.
- All the values of allowed are unique.


In [ ]:
from typing import List
from collections import defaultdict


class Solution:
    def pyramidTransition(self, bottom: str, allowed: List[str]) -> bool:
        """
        Determines whether a pyramid can be built from the given bottom row
        using the allowed triangular patterns.

        Algorithm:
            1. Preprocess the allowed patterns into a mapping where:
               (bottom_left, bottom_right) -> list of possible top blocks.
            2. Use Depth-First Search (DFS) with memoization:
               - At each level, generate all possible next rows.
               - Recursively attempt to build the pyramid upward.
               - Cache failed states to avoid recomputation.
            3. If a row of length 1 is reached, the pyramid is successfully built.

        Args:
            bottom (str): The bottom row of the pyramid.
            allowed (List[str]): List of allowed triangular patterns.

        Returns:
            bool: True if the pyramid can be built, False otherwise.

        Time Complexity:
            O(B^(N*(N-1)/2)) in the worst case,
            where:
                N = length of bottom (≤ 6)
                B = max branching factor (≤ 6)
            Due to memoization and small constraints, this runs efficiently.

        Space Complexity:
            O(N^2 + S),
            where:
                N = maximum recursion depth
                S = number of unique intermediate states stored in memoization.
        """

        # Mapping from bottom pair -> possible top characters
        transitions = defaultdict(list)
        for pattern in allowed:
            transitions[pattern[:2]].append(pattern[2])

        memo = {}

        def dfs(row: str) -> bool:
            """
            Recursively attempts to build the pyramid from the current row.

            Args:
                row (str): Current level of the pyramid.

            Returns:
                bool: True if pyramid can be completed from this row.
            """
            # Base case: top of pyramid reached
            if len(row) == 1:
                return True

            # Return cached result if already computed
            if row in memo:
                return memo[row]

            # Generate all possible next rows
            next_rows = []

            def build_next(index: int, current: str) -> None:
                """
                Backtracking helper to generate all valid next rows.

                Args:
                    index (int): Current index in the row.
                    current (str): Partial next row being built.
                """
                if index == len(row) - 1:
                    next_rows.append(current)
                    return

                pair = row[index:index + 2]
                if pair not in transitions:
                    return

                for ch in transitions[pair]:
                    build_next(index + 1, current + ch)

            build_next(0, "")

            # Try each possible next row
            for next_row in next_rows:
                if dfs(next_row):
                    memo[row] = True
                    return True

            memo[row] = False
            return False

        return dfs(bottom)
